In [ ]:
print("Hello collabski")

Hello collabski


In [ ]:
import torch
import numpy as np

print(torch.__version__)
print(np.__version__)
print(torch.cuda.is_available())

2.6.0+cu124
2.0.2
True


In [ ]:
import torch
import torch.nn as nn
import numpy as np

vocab_size = 6 #6 token test vocabulary
embedding_dim = 128 #vector size per token
word_embedding_layer = nn.Embedding(vocab_size, embedding_dim)

#example:
input_sentence_ids = torch.tensor([[1,3,4,5]], dtype=torch.long)

#move input to gpu
if torch.cuda.is_available():
  print("moving to gpu")
  word_embedding_layer.to('cuda') #moving the whole embed token layer
  input_sentence_ids = input_sentence_ids.to('cuda')
#pass id's through the embedding layer and then get the shape resutls
embedded_sentence = word_embedding_layer(input_sentence_ids)
print(f"sentence ids: {input_sentence_ids.shape}")
print(f"embed sentenc: {embedded_sentence.shape}")
print(f"First word's embedding:{embedded_sentence[0,0, :5]}")
print(f"is tensor in gpu? {embedded_sentence.is_cuda}")


moving to gpu
sentence ids: torch.Size([1, 4])
embed sentenc: torch.Size([1, 4, 128])
First word's embedding:tensor([-1.1455,  0.2516,  0.5178, -0.1356, -0.4431], device='cuda:0',
       grad_fn=<SliceBackward0>)
is tensor in gpu? True


In [ ]:
input_features = embedding_dim #input size of embedding layer
output_featuers = 256 #transforming weights of the layer from 128 number vectors  to 256 number vector
linear_layer = nn.Linear(input_features, output_featuers) #linear changes the weights to find the correct syntax
if torch.cuda.is_available():
  print("moving to gpu")
  linear_layer.to('cuda')

output_linear = linear_layer(embedded_sentence)
print(f"input shape: {embedded_sentence.shape}")
print(f"output shape: {output_linear.shape}")
activation_function = nn.ReLU()
output_activation = activation_function(output_linear)
print(f"{output_activation.shape}")
print(f"{output_activation[0,0, :5]}")
print(f"{output_activation.is_cuda}")



moving to gpu
input shape: torch.Size([1, 4, 128])
output shape: torch.Size([1, 4, 256])
torch.Size([1, 4, 256])
tensor([0.2333, 0.0000, 0.1738, 0.7630, 0.0192], device='cuda:0',
       grad_fn=<SliceBackward0>)
True


In [ ]:
#self attention
import torch
import torch.nn as nn
import torch.nn.functional as F
input_dim = 128
qkv_dim = input_dim
query_linear = nn.Linear(input_dim, qkv_dim, bias = False)
key_linear = nn.Linear(input_dim, qkv_dim, bias = False)
value_linear = nn.Linear(input_dim, qkv_dim, bias = False)

if torch.cuda.is_available():
  print("Moving Q,K, V linear layesr to gpu")
  query_linear.to('cuda')
  key_linear.to('cuda')
  value_linear.to('cuda')

Q = query_linear(embedded_sentence)
K = key_linear(embedded_sentence)
V = value_linear(embedded_sentence)

print(f"\nShape of Q (Query): {Q.shape}")
print(f"Shape of K (Key): {K.shape}")
print(f"Shape of V (Value): {V.shape}")
attention_scores = torch.matmul(Q, K.transpose(-2, -1)) # Shape: [1, 4, 4]

print(f"\nShape of raw attention scores: {attention_scores.shape}")
print("Raw attention scores (example for the single sentence):\n", attention_scores[0].detach().cpu().numpy())
scaling_factor = torch.sqrt(torch.tensor(qkv_dim, dtype=torch.float32))
if torch.cuda.is_available():
    scaling_factor = scaling_factor.to('cuda')
scaled_attention_scores = attention_scores / scaling_factor

print(f"\nShape of scaled attention scores: {scaled_attention_scores.shape}")
print("Scaled attention scores:\n", scaled_attention_scores[0].detach().cpu().numpy())

attention_probabilities = F.softmax(scaled_attention_scores, dim=-1)

print(f"\nShape of attention probabilities: {attention_probabilities.shape}")
print("Attention probabilities (each row sums to 1 for the sentence):\n", attention_probabilities[0].detach().cpu().numpy())
output_attention = torch.matmul(attention_probabilities, V) # Shape: [1, 4, 128]

print(f"\nShape of output from single head self-attention: {output_attention.shape}")
print("Output from self-attention (first word, first 5 numbers, context-rich):\n", output_attention[0, 0, :5].detach().cpu().numpy())
print(f"Is the attention output tensor on GPU? {output_attention.is_cuda}")

Moving Q,K, V linear layesr to gpu

Shape of Q (Query): torch.Size([1, 4, 128])
Shape of K (Key): torch.Size([1, 4, 128])
Shape of V (Value): torch.Size([1, 4, 128])

Shape of raw attention scores: torch.Size([1, 4, 4])
Raw attention scores (example for the single sentence):
 [[ 2.4522426   1.9550277  -1.9412539   0.33083355]
 [-4.0718694   4.869228   -8.965159    6.255979  ]
 [ 1.9293495   4.006773    1.9761771  -1.3404183 ]
 [-7.675651   -3.1479838   6.284437   -0.5502856 ]]

Shape of scaled attention scores: torch.Size([1, 4, 4])
Scaled attention scores:
 [[ 0.21674968  0.17280167 -0.17158423  0.02924183]
 [-0.3599058   0.43038303 -0.7924156   0.5529557 ]
 [ 0.17053202  0.35415205  0.17467102 -0.11847737]
 [-0.6784381  -0.2782451   0.555471   -0.04863884]]

Shape of attention probabilities: torch.Size([1, 4, 4])
Attention probabilities (each row sums to 1 for the sentence):
 [[0.2886659  0.27625433 0.19576913 0.23931056]
 [0.15762049 0.34740075 0.10227655 0.39270222]
 [0.25286293 0.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class MultiHeadSelfAttention(nn.Module):
    def __init__(self, embed_dim, num_heads):
        super().__init__()
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        assert embed_dim % num_heads == 0, "embed_dim must be divisible by num_heads"
        self.head_dim = embed_dim // num_heads
        self.qkv_proj = nn.Linear(embed_dim, embed_dim * 3, bias=False)

        self.out_proj = nn.Linear(embed_dim, embed_dim)

    def forward(self, x):
        batch_size, seq_len, embed_dim = x.shape

        qkv = self.qkv_proj(x)
        q, k, v = qkv.chunk(3, dim=-1)
        q = q.view(batch_size, seq_len, self.num_heads, self.head_dim).transpose(1, 2)
        k = k.view(batch_size, seq_len, self.num_heads, self.head_dim).transpose(1, 2)
        v = v.view(batch_size, seq_len, self.num_heads, self.head_dim).transpose(1, 2)

        print(f"\nInside Attention: Q shape: {q.shape}")
        print(f"Inside Attention: K shape: {k.shape}")
        print(f"Inside Attention: V shape: {v.shape}")

        attention_scores = torch.matmul(q, k.transpose(-2, -1))

        scaled_attention_scores = attention_scores / (self.head_dim ** 0.5)

        attention_probs = F.softmax(scaled_attention_scores, dim=-1)

        output = torch.matmul(attention_probs, v)

        output = output.transpose(1, 2).contiguous().view(batch_size, seq_len, embed_dim)

        output = self.out_proj(output)

        return output


embed_dim_model = 128
num_attention_heads = 8

mha_layer = MultiHeadSelfAttention(embed_dim_model, num_attention_heads)

if torch.cuda.is_available():
    print(f"\nMoving MultiHeadSelfAttention layer to GPU...")
    mha_layer.to('cuda')


print(f"\nInput to Multi-Head Attention Layer shape: {embedded_sentence.shape}")
print(f"Input tensor on GPU? {embedded_sentence.is_cuda}")

output_from_mha = mha_layer(embedded_sentence)

print(f"\nOutput from Multi-Head Attention Layer shape: {output_from_mha.shape}")
print(f"Output tensor on GPU? {output_from_mha.is_cuda}")



Moving MultiHeadSelfAttention layer to GPU...

Input to Multi-Head Attention Layer shape: torch.Size([1, 4, 128])
Input tensor on GPU? True

Inside Attention: Q shape: torch.Size([1, 8, 4, 16])
Inside Attention: K shape: torch.Size([1, 8, 4, 16])
Inside Attention: V shape: torch.Size([1, 8, 4, 16])

Output from Multi-Head Attention Layer shape: torch.Size([1, 4, 128])
Output tensor on GPU? True


In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class MultiHeadSelfAttention(nn.Module):
    def __init__(self, embed_dim, num_heads):
        super().__init__()
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        assert embed_dim % num_heads == 0, "embed_dim must be divisible by num_heads"
        self.head_dim = embed_dim // num_heads
        self.qkv_proj = nn.Linear(embed_dim, embed_dim * 3, bias=False)
        self.out_proj = nn.Linear(embed_dim, embed_dim)

    def forward(self, x):
        bsz, seq_len, _ = x.shape
        qkv = self.qkv_proj(x)                        # [b, seq, 3*embed]
        q, k, v = qkv.chunk(3, dim=-1)                # each [b, seq, embed]
        q = q.view(bsz, seq_len, self.num_heads, self.head_dim).transpose(1, 2)
        k = k.view(bsz, seq_len, self.num_heads, self.head_dim).transpose(1, 2)
        v = v.view(bsz, seq_len, self.num_heads, self.head_dim).transpose(1, 2)

        scores = torch.matmul(q, k.transpose(-2, -1))
        scores = scores / (self.head_dim ** 0.5)
        probs = F.softmax(scores, dim=-1)
        context = torch.matmul(probs, v)              # [b, heads, seq, head_dim]

        context = context.transpose(1, 2).contiguous().view(bsz, seq_len, self.embed_dim)
        out = self.out_proj(context)
        return out


class TransformerBlock(nn.Module):
    def __init__(self, embed_dim, num_heads, ff_dim, dropout_rate=0.1):
        super().__init__()
        self.attention = MultiHeadSelfAttention(embed_dim, num_heads)
        self.norm1 = nn.LayerNorm(embed_dim)
        self.feed_forward = nn.Sequential(
            nn.Linear(embed_dim, ff_dim),
            nn.ReLU(),
            nn.Linear(ff_dim, embed_dim)
        )
        self.norm2 = nn.LayerNorm(embed_dim)
        self.dropout = nn.Dropout(dropout_rate)

    def forward(self, x):
        y = self.norm1(x)
        y = self.attention(y)
        x = x + self.dropout(y)

        y = self.norm2(x)
        y = self.feed_forward(y)
        x = x + self.dropout(y)
        return x


if __name__ == "__main__":
    embed_dim_model = 128
    num_attention_heads = 8
    ff_hidden_dim = embed_dim_model * 4

    transformer_block = TransformerBlock(
        embed_dim=embed_dim_model,
        num_heads=num_attention_heads,
        ff_dim=ff_hidden_dim
    )

    batch_size, seq_len = 2, 10
    embedded_sentence = torch.randn(batch_size, seq_len, embed_dim_model)

    if torch.cuda.is_available():
        print("Moving TransformerBlock to GPU…")
        transformer_block = transformer_block.cuda()
        embedded_sentence = embedded_sentence.cuda()

    print("Input to Transformer Block shape:", embedded_sentence.shape)
    print("Input tensor on GPU?", embedded_sentence.is_cuda)

    output_from_transformer_block = transformer_block(embedded_sentence)

    print("Output from Transformer Block shape:", output_from_transformer_block.shape)
    print("Output tensor on GPU?", output_from_transformer_block.is_cuda)



Moving TransformerBlock to GPU…
Input to Transformer Block shape: torch.Size([2, 10, 128])
Input tensor on GPU? True
Output from Transformer Block shape: torch.Size([2, 10, 128])
Output tensor on GPU? True
